<a href="https://colab.research.google.com/github/divij-dawar/Fashion_MNIST/blob/main/Inception_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing necessary libraries
import time
import torch
import numpy as np
import torch.nn as nn
from torch.optim import Adam
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import Tensor

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.CIFAR10(root='./data',train=True,download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data',train=False,download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

In [ ]:
#Classes
labels = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
#Loading GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [ ]:
# Convolutional Block of the Model
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels,kernel_size,stride, padding, **kwargs):
    super(ConvBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,**kwargs)
    self.bn = nn.BatchNorm2d(out_channels,eps=0.001)
    self.activation = nn.ReLU()

  def forward(self,x: Tensor) -> Tensor :
      x = self.conv(x)
      x = self.bn(x)
      x = self.activation(x)

      return x

In [ ]:
# Inception Block of the Model
class InceptionBlock(nn.Module):
  def __init__(self,in_channels: int, filters_1x1, filters_3x3, filters_5x5,
               reduced_filters_3x3, reduced_filters_5x5,out_channels):
    super(InceptionBlock, self).__init__()
    self.branch1 = nn.Sequential(
        ConvBlock(in_channels, filters_1x1, kernel_size=1, stride=1, padding=0)
    )

    self.branch2 = nn.Sequential(
        ConvBlock(in_channels, reduced_filters_3x3, kernel_size=1, stride=1,padding=0),
        ConvBlock(reduced_filters_3x3, filters_3x3, kernel_size=3, stride=1,padding=1)
    )

    self.branch3 = nn.Sequential(
        ConvBlock(in_channels, reduced_filters_5x5, kernel_size=1,stride=1, padding=0),
        ConvBlock(reduced_filters_5x5, filters_5x5, kernel_size=5,stride=1, padding=2)
    )

    self.branch4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
        ConvBlock(in_channels, out_channels, kernel_size=1, stride=1,padding=0)
    )

  def forward(self, x: torch.Tensor)-> torch.Tensor:
    out1 = self.branch1(x)
    out2 = self.branch2(x)
    out3 = self.branch3(x)
    out4 = self.branch4(x)

    output = torch.cat([out1, out2, out3, out4], 1)
    return output

In [ ]:
# Auxiliary Netowork Architecture
class AuxiliaryClassifier(nn.Module):
  def __init__(self,in_channels):
    super(AuxiliaryClassifier,self).__init__()
    self.Adaptive_Avg = nn.AdaptiveAvgPool2d(output_size=(4,4))
    self.conv = nn.Conv2d(in_channels,out_channels=128,kernel_size=(1,1),stride=(1,1),padding=(0,0))
    self.activation = nn.ReLU()

    self.fc1 = nn.Linear(in_features=2048,out_features=1024)
    self.DropOut = nn.Dropout(0.7)
    self.fc2 = nn.Linear(in_features=1024,out_features=10)

  def forward(self,x: Tensor):
    x = self.Adaptive_Avg(x)
    x = self.conv(x)
    x = self.activation(x)

    x = torch.flatten(x,1)

    x = self.fc1(x)
    x = self.activation(x)
    x = self.DropOut(x)
    x = self.fc2(x)

    return x

In [ ]:
# Model Architecture
class NeuralNetwork(nn.Module):
  def __init__ (self,num_classes=10):
    super(NeuralNetwork,self).__init__()

    self.conv1 = ConvBlock(3,64,kernel_size=7,stride=2,padding=3)
    self.pooling1 = nn.MaxPool2d(3,stride=2,padding=0,ceil_mode=True)
    self.conv2 = ConvBlock(64,64, kernel_size=1, stride=1, padding=0)
    self.conv3 = ConvBlock(64,192, kernel_size=3, stride=1, padding=1)
    self.pooling2 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)

    self.inception3A = InceptionBlock(in_channels=192,
                                      filters_1x1 = 64,
                                      reduced_filters_3x3=96,
                                      filters_3x3 = 128,
                                      reduced_filters_5x5 = 16,
                                      filters_5x5 = 32,
                                      out_channels = 32)

    self.inception3B = InceptionBlock(in_channels=256,
                                      filters_1x1 = 128,
                                      reduced_filters_3x3=128,
                                      filters_3x3=192,
                                      reduced_filters_5x5=32,
                                      filters_5x5=96,
                                      out_channels=64)

    self.pooling4 = nn.MaxPool2d(3,stride=2,padding=0,ceil_mode=True)

    self.inception4A = InceptionBlock(in_channels=480,
                                      filters_1x1=192,
                                      reduced_filters_3x3=96,
                                      filters_3x3=208,
                                      reduced_filters_5x5 = 16,
                                      filters_5x5 = 48,
                                      out_channels=64)

    self.inception4B = InceptionBlock(in_channels=512,
                                        filters_1x1=160,
                                        reduced_filters_3x3=112,
                                        filters_3x3=224,
                                        reduced_filters_5x5=24,
                                        filters_5x5=64,
                                        out_channels=64)

    self.inception4C = InceptionBlock(in_channels=512,
                                      filters_1x1=128,
                                      reduced_filters_3x3=128,
                                      filters_3x3=256,
                                      reduced_filters_5x5 = 24,
                                      filters_5x5=64,
                                      out_channels=64)

    self.inception4D = InceptionBlock(in_channels=512,
                                      filters_1x1=112,
                                      reduced_filters_3x3=144,
                                      filters_3x3=288,
                                      reduced_filters_5x5 = 32,
                                      filters_5x5=64,
                                      out_channels=64)

    self.inception4E = InceptionBlock(in_channels=528,
                                      filters_1x1=256,
                                      reduced_filters_3x3=160,
                                      filters_3x3=320,
                                      reduced_filters_5x5 = 32,
                                      filters_5x5=128,
                                      out_channels=128)

    self.pooling5 = nn.MaxPool2d(3,stride=1,padding=1,ceil_mode=True)

    self.inception5A = InceptionBlock(in_channels=832,
                                      filters_1x1=256,
                                      reduced_filters_3x3=160,
                                      filters_3x3=320,
                                      reduced_filters_5x5 = 32,
                                      filters_5x5=128,
                                      out_channels=128)

    self.inception5B = InceptionBlock(in_channels=832,
                                      filters_1x1=384,
                                      reduced_filters_3x3=192,
                                      filters_3x3=284,
                                      reduced_filters_5x5 = 48,
                                      filters_5x5=128,
                                      out_channels=128
                                      )

    self.pool6 = nn.AdaptiveAvgPool2d((1,1))
    self.dropout = nn.Dropout(0.4)
    self.fc = nn.Linear(1024,num_classes)

    self.auxiliaryA = AuxiliaryClassifier(512)
    self.auxiliaryB = AuxiliaryClassifier(528)

  def forward(self,x):
    x = self.conv1(x)
    x = self.pooling1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.pooling2(x)
    x = self.inception3A(x)
    x = self.inception3B(x)
    x = self.pooling4(x)
    x = self.inception4A(x)

    aux1 = self.auxiliaryA(x)

    x = self.inception4B(x)
    x = self.inception4C(x)
    x = self.inception4D(x)

    aux2 = self.auxiliaryB(x)

    x = self.inception4E(x)
    x = self.pooling5(x)
    x = self.inception5A(x)
    x = self.inception5B(x)
    x = self.pool6(x)
    x = torch.flatten(x,1)
    x = self.dropout(x)
    x = self.fc(x)

    return x, aux1, aux2

InceptionNetwork = NeuralNetwork()
InceptionNetwork = InceptionNetwork.to(device)

In [ ]:
#Setting the Hyperparameters
optimizer = Adam(InceptionNetwork.parameters(),lr =0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 30

#training loop
for e in range(epochs):

  correct_prediction = 0
  running_loss = 0.0
  total_images = 0

  InceptionNetwork.train()

  for images,labels in train_loader:
    images, labels = images.to(device) , labels.to(device)

    prediction,aux_pred1, aux_pred2 = InceptionNetwork(images)
    network_loss = criterion(prediction,labels)
    aux_loss1 = criterion(aux_pred1, labels)
    aux_loss2 = criterion(aux_pred2, labels)

    main_loss = network_loss + (0.3 * aux_loss1) + (0.3 * aux_loss2)

    optimizer.backward()
    optimizer.zero_grad()
    optimizer.step()

    running_loss += main_loss.item()

    _, predicted = torch.max(prediction,1)
    correct_prediction += (predicted==labels).sum().item()
    total_images += labels.size(0)

  epoch_loss = running_loss / len(train_loader)
  epoch_accuracy = correct_prediction / total_images

  print(f"Epoch {e+1}/{epochs}, Loss: {epoch_loss: .4f}, Accuracy : {epoch_accuracy: 3.f}")

In [ ]:
nb_test_examples = 10000
correct = 0

InceptionNetwork.eval().cuda()

#Training loop
with  torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        prediction, _, _ = InceptionNetwork(inputs)


        _, predicted_class = torch.max(prediction.data, 1)


        correct += (predicted_class == labels).float().sum().item()

test_accuracy = correct / nb_test_examples
print('Test accuracy: {}'.format(test_accuracy))